<a href="https://colab.research.google.com/github/sp2005-im/EmoMap--Emotion-Detection-Hackathon/blob/main/EmoMap%20Model/EmoMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#loading of data from csv files
import cv2
import numpy as np
import os
import pandas as pd
import plotly.express as px
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization,MaxPooling2D,Activation,Input
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import matplotlib.pyplot as plt

In [ ]:
#x_train
# Read the CSV file
filepath1 = '../train_dataset.csv'
filepath2 = '../test_dataset.csv'


#df1 = pd.read_csv(filepath1)
df2 = pd.read_csv(filepath2)
df1 = pd.read_csv(filepath1)
print(df1.shape)

(35887, 3)


In [ ]:
import os

# Get the current working directory
current_directory = os.getcwd()

# List all files and directories in the current working directory
files = os.listdir(current_directory)

# Print the list of files and directories
print("Files and directories in '", current_directory, "':")
for file in files:
    print(file)

In [ ]:
#data preprocessing

CLASS_LABELS  = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise','Neutral']
df1 = df1.sample(frac=1)
labels = to_categorical(df1[['emotion']], num_classes=7)

#conversion into numpy array

train_pixels = df1["pixels"].astype(str).str.split(" ").tolist()
train_pixels = np.uint8(train_pixels)

pixels = train_pixels.reshape((5000*2304,1))
scaler = StandardScaler()
pixels = scaler.fit_transform(pixels)

pixels = train_pixels.reshape((5000, 48, 48,1))

x_train = pixels
y_train = labels


#splitting into validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=False)

In [ ]:
#data augmentation

datagen = ImageDataGenerator(  width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               horizontal_flip = True,
                               zoom_range = 0.2)
valgen = ImageDataGenerator(   width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               horizontal_flip = True,
                               zoom_range = 0.2)

datagen.fit(x_train)
valgen.fit(x_val)

train_generator = datagen.flow(x_train, y_train, batch_size=32)
val_generator = datagen.flow(x_val, y_val, batch_size=32)

In [ ]:
#designing the model
def cnn_model():

    model= tf.keras.models.Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48,1)))
    model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(512,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Dense(7, activation='softmax'))
    model.compile(
        optimizer = Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
model = cnn_model()

model.summary()

checkpointer = [EarlyStopping(monitor = 'val_accuracy', verbose = 1,
                              restore_best_weights=True,mode="max",patience = 5),
                ModelCheckpoint('best_model.keras',monitor="val_accuracy",verbose=1,
                                save_best_only=True,mode="max")]

history = model.fit(train_generator,
                    epochs=50,
                    batch_size=32,
                    verbose=1,
                    callbacks=[checkpointer],
                    validation_data=val_generator)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 48, 48, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 48, 48, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 48, 48, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 24, 24, 128)         │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 24, 24, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 12, 12, 512)         │         590,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 12, 12, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 6, 6, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 6, 6, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 3, 3, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 3, 3, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │              

 Total params: 5,810,183 (22.16 MB)

 Trainable params: 5,805,191 (22.15 MB)

 Non-trainable params: 4,992 (19.50 KB)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1009/1010 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1804 - loss: 14.3737
Epoch 1: val_accuracy improved from -inf to 0.23572, saving model to best_model.keras
1010/1010 ━━━━━━━━━━━━━━━━━━━━ 59s 41ms/step - accuracy: 0.1804 - loss: 14.3723 - val_accuracy: 0.2357 - val_loss: 12.0805
Epoch 2/50
1009/1010 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2222 - loss: 11.4353
Epoch 2: val_accuracy improved from 0.23572 to 0.25690, saving model to best_model.keras
1010/1010 ━━━━━━━━━━━━━━━━━━━━ 29s 28ms/step - accuracy: 0.2222 - loss: 11.4337 - val_accuracy: 0.2569 - val_loss: 8.9006
Epoch 3/50
1010/1010 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2307 - loss: 8.3156
Epoch 3: val_accuracy improved from 0.25690 to 0.28142, saving model to best_model.keras
1010/1010 ━━━━━━━━━━━━━━━━━━━━ 29s 28ms/step - accuracy: 0.2307 - loss: 8.3149 - val_accuracy: 0.2814 - val_loss: 6.1342
Epoch 4/50
1008/1010 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2577 - loss: 5.6996
Epoch 4: val_accuracy i

In [ ]:
#conversion into numpy array

test_pixels = df2["pixels"].astype(str).str.split(" ").tolist()
test_pixels = np.uint8(test_pixels)

pixels = test_pixels.reshape((2500*2304,1))
scaler = StandardScaler()
pixels = scaler.fit_transform(pixels)

pixels = test_pixels.reshape((2500, 48, 48,1))

x_test = pixels
print(x_test.shape)

(2500, 48, 48, 1)


In [ ]:
preds = model.predict(x_test)
y_pred = np.argmax(preds , axis = 1 )
print(y_pred)

79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step
[1 0 5 ... 0 3 4]


In [ ]:
test_ids = df2["id"]

submission_df = pd.DataFrame({
    'id': test_ids,  # Your test sample IDs
    'emotion': y_pred  # Your model's predictions
})
submission_df.to_csv('../submission.csv', index=False)

In [ ]:
filepath = '../submission.csv'
df = pd.read_csv(filepath)